In [1]:
import examples, core 
import os
import yaml
import dspy
import metrics

In [2]:
with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

llm = dspy.OpenAI(model='gpt-4o', api_key=openai_api_key, max_tokens=2000)

In [3]:
data = examples.load_examples("examples.json")

exp_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.conciseness,
                metrics.context_awareness,
            ], verbose=1
        )

explingo = core.Explingo(llm=llm, context="The model predicts house prices", 
                         examples=data, metric=exp_metrics)

In [4]:
test_explanations = [d.explanation for d in data]
test_explanation_format = data[0].explanation_format

gold_standards = [d for d in data if hasattr(d, "narrative")]
max_optimal_length = max([len(d.narrative) for d in gold_standards])
max_optimal_length

474

In [5]:
VERIFY = True

if VERIFY:
    ver_metrics = metrics.Metrics(
            [
                metrics.accuracy,
                metrics.fluency,
                metrics.conciseness,
            ], 
        verbose=0, 
        metric_kwargs={"conciseness": {"max_optimal_length": max_optimal_length}}
    )
    print(ver_metrics(gold_standards[0], gold_standards[0]))

(5.0, accuracy       2.0
fluency        1.0
conciseness    2.0
dtype: float64)


# BASIC PROMPTING

TODO: update Explingo's prompting method to take in multiple different prompt options. 
DSPy does not support this by default, but we are using DSPy's evaluations

In [6]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="basic", max_iters=5)

Narrative: The model predicts the house price based on several key features. The above ground living area of 1256 square feet decreases the predicted price by $12,527.46. The overall material and finish rating of the house, which is 5, reduces the price by $10,743.76. The absence of a second floor contributes to a decrease of $10,142.29. The house's location within the Edwards neighborhood in Ames city limits lowers the price by $9,913.81. However, the presence of a wood deck area of 736 square feet increases the predicted price by $9,846.38.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 854.00, contributes significantly to increasing the price by 12757.84 units. The original construction date of the house, which is 2003.00, also adds a substantial amount of 9115.72 units to the price. On the other hand, the total square feet 

(6.964,
 accuracy             2.000
 fluency              1.000
 conciseness          1.964
 context_awareness    2.000
 dtype: float64)

In [7]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="few-shot", max_iters=5)

Narrative: The relatively smaller above ground living space reduced the predicted price by about $12,500. The house's average material and finish quality (rated 5/10) reduced the price by about $10,700. The absence of a second floor reduced the price by about $10,100. The house's location in the Edwards neighborhood reduced the price by about $9,900. The relatively larger wood deck area increased the price by about $9,800.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The house's larger than average second floor increased the predicted price by about $12,000. The house is newer than average, with a construction year of 2003, which increased the price by about $9,000. The relatively smaller basement and first floor areas reduced the price by about $6,000 and $5,000, respectively. The house's location in CollgCr reduced the price by about $5,000.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0

(7.0,
 accuracy             2.0
 fluency              1.0
 conciseness          2.0
 context_awareness    2.0
 dtype: float64)

In [8]:
explingo.run_experiment(test_explanations, test_explanation_format, prompt_type="bootstrap-few-shot", max_iters=5)

  0%|          | 0/25 [00:00<?, ?it/s]

Narrative: The model predicts the house price based on several key features. The above ground living area, which is 1256 square feet, decreases the predicted price by $12,527.46. The overall material and finish rating of the house, which is 5, reduces the price by $10,743.76. The absence of a second floor further decreases the price by $10,142.29. The house's location within the Edwards neighborhood in Ames city limits lowers the price by $9,913.81. However, the presence of a wood deck area of 736 square feet increases the predicted price by $9,846.38.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 854.00, contributes positively with a value of 12757.84. The original construction date of the house, which is 2003.00, also adds positively to the price with a contribution of 9115.72. However, the total square feet of the basement 

 12%|█▏        | 3/25 [00:00<00:01, 21.47it/s]

Narrative: The model predicts the house price based on several key features. The second floor square feet, which is 866.00, contributes significantly to increasing the price by 13079.62. The original construction date of the house, which is 2001.00, also adds a substantial amount of 8500.21 to the price. The above ground living area square feet, measured at 1786.00, further increases the price by 5844.30. However, the physical location within Ames city limits, specifically in the CollgCr area, decreases the price by 4761.42. Additionally, the total square feet of the basement area, which is 920.00, reduces the price by 4747.08.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The house price is influenced by several factors. The original construction date of 1915 decreases the price by $17,966.77. Being located within the Crawfor area of Ames city increases the price by $17,703.26. The second floor, with 756 square feet, adds $10,12

 24%|██▍       | 6/25 [00:00<00:00, 19.90it/s]

...

 88%|████████▊ | 22/25 [00:01<00:00, 20.52it/s]

Narrative: The model predicts that the student will pass their class primarily because they have no past class failures, which strongly contributes to the positive prediction. Additionally, attending the school labeled "GP" and being 18 years old also positively influence the prediction. The reason for choosing the school and living in an urban area have smaller, but still positive, contributions to the prediction.
Total Score: 7.0
accuracy: 2.0, fluency: 1.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The model predicts that the student will pass their class. The most significant positive factor is that the student has no past class failures, which contributes strongly to the likelihood of passing. Attending the school labeled "MS" has a negative impact on the prediction. Choosing the school for its course offerings also slightly decreases the likelihood of passing. The father's job being categorized as "other" and the student's desire to pursue higher education both hav

100%|██████████| 25/25 [00:01<00:00, 20.15it/s]


Bootstrapped 3 full traces after 25 examples in round 0.
Narrative: The model predicts that the house price will be lower. The above ground living area square feet significantly contributes to this lower prediction. Additionally, the overall material and finish of the house, rated at 5, also negatively influences the prediction. The absence of a second floor and the house's location within the Edwards area of Ames city further contribute to the lower price prediction. However, the presence of a wood deck area positively impacts the house price, though this effect is not enough to offset the other negative contributions.
Total Score: 8.0
accuracy: 2.0, fluency: 2.0, conciseness: 2.0, context_awareness: 2.0, 
--
Narrative: The model predicts the house price to be higher primarily due to the large second floor square footage, which significantly contributes to the price increase. Additionally, the house's original construction date in 2003 also positively influences the price. However, th

(7.4,
 accuracy             2.0
 fluency              1.4
 conciseness          2.0
 context_awareness    2.0
 dtype: float64)